# Introduction

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import numpy as np
import math
import sys

In [3]:
import matplotlib.pyplot as plt

# Prepare dataset
## As an example we consider $y = sin(x)$, with $x \sim \mathcal{N}_{0, 0.5}$ and noisy $y$

In [42]:
n_points = 50

x = 2*np.pi * np.random.normal(0, 0.5, n_points)
x.sort()
x = x.reshape(n_points, 1)
y = np.sin(x) + np.random.normal(-0.3, 0.3, x.shape)

# Plot to visualise target function
plt.scatter(x,y)

# Convert to tensor for model training
data = torch.Tensor(np.concatenate((x,y),-1))

# Define Bayesian linear layer
## Here we introduce a simple linear layer based on variational inference

In [12]:
class VBLinear(nn.Module):
    def __init__(self, in_features, out_features, prior_prec=1.0, map=True):
        super(VBLinear, self).__init__()
        self.n_in = in_features
        self.n_out = out_features

        self.prior_prec = prior_prec
        self.map = map

        self.bias = nn.Parameter(torch.Tensor(out_features))
        self.mu_w = nn.Parameter(torch.Tensor(out_features, in_features))
        self.logsig2_w = nn.Parameter(torch.Tensor(out_features, in_features))
        
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.mu_w.size(1))
        self.mu_w.data.normal_(0, stdv)
        self.logsig2_w.data.zero_().normal_(-9, 0.001)
        self.bias.data.zero_()
       
    # Define KL divergence between the predicted and the variational posteriors
    def KL(self):
            logsig2_w = self.logsig2_w.clamp(-11, 11)
            kl = 0.5 * (self.prior_prec * (self.mu_w.pow(2) + logsig2_w.exp()) - logsig2_w - 1 - np.log(self.prior_prec)).sum()
            return kl

    def forward(self, input):
        if self.training:
            mu_out = F.linear(input, self.mu_w, self.bias)
            logsig2_w = self.logsig2_w.clamp(-11, 11)
            s2_w = logsig2_w.exp()
            var_out = F.linear(input.pow(2), s2_w) + 1e-8
            return mu_out + var_out.sqrt() * torch.randn_like(mu_out)
        else:
            if self.map:
                return F.linear(input, self.mu_w, self.bias)
            else:
                mu_out = F.linear(input, self.mu_w, self.bias)
                logsig2_w = self.logsig2_w.clamp(-11, 11)
                s2_w = logsig2_w.exp()
                var_out = F.linear(input.pow(2), s2_w) + 1e-8
                return mu_out + var_out.sqrt() * torch.randn_like(mu_out)


# Build model

In [ ]:
class Regressor(nn.Module):
    def __init__(self, n_layers, input_dim, hidden_dim):
        super().__init__()
        self.n_layers   = n_layers
        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim
    
    def build_model(self):
        model = nn.ModuleList()
        
        model.append(VBLinear(self.input_dim, self.hidden_dim))
        model.append(nn.ReLU())
        
        for l in range(self.n_layers):
            model.append(VBLinear(self.hidden_dim, self.hidden_dim))
            model.append(nn.ReLU())
            
        model.append(VBLinear(self.hidden_dim, 2))
        
        self.model = model
        self.params_trainable = list(filter(lambda p: p.requires_grad, self.parameters()))
            
    def forward(self, x):
        for layer in self.model:
            x = layer(x)
        return x
    
    def set_optimizer(self):
        lr = 1e-3
        eps = 1e-6
        weight_decay = 0.
        gamma = 0.99
        
        self.optim = torch.optim.Adam(self.params_trainable, 
                                      lr = lr, 
                                      betas=[0.9,0.999],
                                      eps = eps,
                                      weight_decay = weight_decay)
        self.scheduler = torch.optim.lr_scheduler.StepLR(optimizer=self.optim,
                                                         step_size=1000,
                                                         gamma=gamma)
            

# Create model

In [49]:
regressor = Regressor(n_layers=0, input_dim=1, hidden_dim=5)
regressor.build_model()
regressor.set_optimizer()

# Print model summary
print(regressor)
print('Total parameters: %d' % sum([np.prod(p.size()) for p in regressor.params_trainable]))

Regressor(
  (model): ModuleList(
    (0): VBLinear()
    (1): ReLU()
    (2): VBLinear()
  )
)
Total parameters: 37


In [8]:
# Train model

In [54]:
print('Epoch\t\tNLL train\tKL \t\t lr')
try:
    for epoch in range(50000):
        
        pred = regressor(data[:,0].view(n_points,1))
        
        target = data[:,1]
        
        mu_pred    = pred[:,0]
        sigma_pred = pred[:,1]
        
        # Compute neg log likelihood on the given dataset
        nll = torch.mean(0.5 * torch.exp(-sigma_pred) * (mu_pred - target)**2 + 0.5 * sigma_pred)
        
        # Add KL divergence
        kl = 0.
        for layer in regressor.model:
            if hasattr(layer, 'KL'):
                kl += layer.KL()
        kl /= n_points
        
        nll += kl
        
        regressor.optim.zero_grad()
        nll.backward()
        regressor.optim.step()
        regressor.scheduler.step()
        
        if not epoch % 1000:
            print('%.3i \t\t%.6f \t%.6f\t %.2e' % (epoch,
                                                nll.item(),
                                                kl.item(),
                                                regressor.optim.param_groups[0]['lr']), flush=True)
except:
    raise

Epoch		NLL train	KL 		 lr
000 		-0.055762 	0.472697	 4.05e-03
1000 		-0.058214 	0.468443	 4.01e-03
2000 		-0.022371 	0.470429	 3.97e-03
3000 		0.190856 	0.474204	 3.93e-03
4000 		-0.077904 	0.473172	 3.89e-03
5000 		-0.114509 	0.474070	 3.85e-03
6000 		-0.065016 	0.472764	 3.81e-03
7000 		0.061024 	0.482370	 3.77e-03
8000 		-0.009910 	0.486673	 3.73e-03
9000 		0.021512 	0.483949	 3.70e-03
10000 		0.075907 	0.485092	 3.66e-03
11000 		-0.045027 	0.479089	 3.62e-03
12000 		-0.037692 	0.477551	 3.59e-03
13000 		-0.112576 	0.475547	 3.55e-03
14000 		-0.062126 	0.474448	 3.52e-03
15000 		-0.084368 	0.470239	 3.48e-03
16000 		-0.013938 	0.471952	 3.45e-03
17000 		0.097094 	0.478468	 3.41e-03
18000 		0.034236 	0.485024	 3.38e-03
19000 		-0.056372 	0.479951	 3.34e-03
20000 		-0.070693 	0.477305	 3.31e-03
21000 		0.013649 	0.475000	 3.28e-03
22000 		-0.075687 	0.475141	 3.24e-03
23000 		-0.128030 	0.476184	 3.21e-03
24000 		-0.106564 	0.476121	 3.18e-03
25000 		-0.135467 	0.475846	 3.15e-03
2600

In [52]:
pred = regressor(torch.Tensor(x).view(n_points,1)).detach().numpy()
mu_pred = pred[:,0]
sigma_pred = np.exp(pred[:,1])**0.5

x = x.flatten()
y = y.flatten()

plt.scatter(x, y, label='Training data')
plt.plot(x, mu_pred, color='C1', label='Predicted')
plt.fill_between(x, mu_pred - sigma_pred , mu_pred + sigma_pred, alpha=0.2, color='C1')
plt.legend()